# Open Tools for Urban, Regional, and Neighborhood Science

**A Gentle Introduction to `geosnap` for Neighborhood Analytics**

https://github.com/oturns/geosnap

### Resources:

- [API Docs](https://oturns.github.io/geosnap/api.html)
- [Tutorial Materials](https://github.com/knaaptime/pysal-narsc22/)
- [Tutorial Video](https://www.youtube.com/watch?v=4AHJVMs7iH4)

`geosnap` - the geospatial neighborhood analysis package - provides a suite of tools for understanding the composition and extent of [endogenous] neighborhoods and regions in a study area. It provides: 

 - simple access to commonly-used datasets in urban and regional analyses (https://open.quiltdata.com/b/spatial-ucr)
     - demographic data (Census/ACS)
     - employment (LEHD)
     - environment (EPA)
     - travel infrastructure (OSM)
     - public education systems (NCES)
     - educational acievement (SEDA)
     - land use/land cover (NLCD)

 - an easy interface to build geodemographic typologies
     - classic aspatial typologies
     - constrained homogenous regions

 - built-in functionality to facilitate spatiotemporal analysis
     - within time-period standardization
     - boundary harmonization
     - inflation adjustment

 - bespoke plotting tools to help visualize neighborhood dynamics
    - temporally-static choropleth mapping
    - animated mapping

 - state-of-the-art techniques for modeling neighborhood change over time
     - spatial Markov transition models
     - sequence analysis

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from geosnap import DataStore
from geosnap.io import get_acs, get_census, get_nces

## Datasets

In [ ]:
datasets = DataStore()

In [ ]:
dir(datasets)

In [ ]:
datasets.show_data_dir()

In [ ]:
msas = datasets.msas()
msas

In [ ]:
msas = msas[msas['type'].str.startswith('Metro')]

In [ ]:
msas.plot()

In [ ]:
msas.head(50).explore()

In [ ]:
# SF and Alameda counties

counties = ["06075", "06001"]

In [ ]:
get_acs?

In [ ]:
acs_teens = get_acs(
    datasets, county_fips=counties, years=[2016, 2017, 2018], level="tract"
)

In [ ]:
acs_teens.head()

In [ ]:
acs_teens[acs_teens.year == 2018].explore(
    "p_nonhisp_white_persons",
    scheme="quantiles",
    tiles="CartoDB Positron",
    cmap="GnBu",
    tooltip=["p_nonhisp_white_persons", "geoid"],
)

In [ ]:
acs_teens.year.unique()

In [ ]:
acs_teens[acs_teens.year == 2018].shape

In [ ]:
for year in acs_teens.year.unique():
    print(acs_teens[acs_teens.year == year].shape[0])

In [ ]:
census = get_census(datasets, county_fips=counties)

In [ ]:
census.year.unique()

In [ ]:
for year in census.year.unique():
    print(census[census.year == year].shape[0])

## Standardizing Boundaries Over Time

In [ ]:
from geosnap.harmonize import harmonize

In [ ]:
harmonize?

In [ ]:
columns = [
    "median_household_income",
    "median_home_value",
    "p_asian_persons",
    "p_hispanic_persons",
    "p_nonhisp_black_persons",
    "p_nonhisp_white_persons",
]

In [ ]:
census = harmonize(
    census.set_index("geoid"),
    target_year=2010,
    intensive_variables=columns,
    extensive_variables=["n_total_pop"],
)

In [ ]:
census = census.reset_index()

In [ ]:
for year in census.year.unique():
    print(census[census.year == year].shape[0])

In [ ]:
census[census.year == 1990].explore(
    "p_nonhisp_white_persons", scheme="quantiles", k=6, cmap="GnBu"
)

## Geodemographic Neighborhood Types

In [ ]:
from geosnap.analyze import cluster, transition
from geosnap.visualize import (
    animate_timeseries,
    plot_timeseries,
    plot_transition_matrix,
)

In [ ]:
import seaborn as sns

In [ ]:
sns.set_style(None)

In [ ]:
acs18 = acs_teens[acs_teens.year == 2018]

In [ ]:
for col in columns:
    ax = acs18.plot(
        col,
        scheme="quantiles",
        k=6,
    )
    ax.set_title(col)

In [ ]:
sf18_kmeans = cluster(acs18, columns=columns, method="kmeans", n_clusters=6)

In [ ]:
sf18_kmeans.head()

In [ ]:
from geosnap import visualize as gvz

In [ ]:
sf18_kmeans.explore(
    "kmeans", categorical=True, tiles="CartoDB Positron", tooltip=columns
)

changing the number of number of clusters

In [ ]:
sf18_kmeans10 = cluster(acs18, columns=columns, method="kmeans", n_clusters=10)

In [ ]:
sf18_kmeans10.explore("kmeans")

In [ ]:
gvz.plot_violins_by_cluster?

In [ ]:
sns.set_style("whitegrid")

In [ ]:
gvz.plot_violins_by_cluster(
    sf18_kmeans,
    columns,
    cluster_col="kmeans",
    violin_kwargs={"palette": "Set1"},
    nrows=3,
    ncols=2,
)

In [ ]:
gvz.plot_violins_by_cluster(
    sf18_kmeans10,
    columns,
    cluster_col="kmeans",
    violin_kwargs={"palette": "Set1"},
    nrows=3,
    ncols=2,
)

changing the clustering algorithm

In [ ]:
sf18_ward = cluster(acs18, columns=columns, method="ward", n_clusters=6)

In [ ]:
sf18_ward.explore("ward", categorical=True, tiles="CartoDB Positron", tooltip=columns)

In [ ]:
_, kmeans_mod = cluster(
    acs18, columns=columns, method="kmeans", n_clusters=6, return_model=True
)

_, ward_mod = cluster(
    acs18, columns=columns, method="ward", n_clusters=6, return_model=True
)

In [ ]:
kmeans_mod.silhouette_score

In [ ]:
ward_mod.silhouette_score

So according to the silhouette score, the kmeans model fits better

In [ ]:
sf18_ward = cluster(
    acs18,
    columns=columns,
    method="ward",
    n_clusters=6,
)

In [ ]:
sf18_kmeans10.explore(
    "kmeans", categorical=True, tiles="CartoDB Positron", tooltip=columns
)

In [ ]:
from geosnap.analyze import regionalize

In [ ]:
sf_regions = regionalize(acs18, columns=columns, method="ward_spatial", n_clusters=10)

In [ ]:
sf_regions.explore(
    "ward_spatial", categorical=True, tiles="CartoDB Positron", tooltip=columns
)

In [ ]:
gvz.plot_violins_by_cluster(sf_regions, columns, "ward_spatial", nrows=3, ncols=2)

## Modeling Neighborhood Change

In [ ]:
sf_types_kmeans = cluster(acs_teens, columns=columns, method="kmeans", n_clusters=6)

In [ ]:
plot_timeseries?

In [ ]:
import contextily as ctx

In [ ]:
plot_timeseries(
    sf_types_kmeans,
    column="median_home_value",
    scheme="quantiles",
    k=6,
    ncols=3,
    nrows=1,
    figsize=(15, 5),
    ctxmap=ctx.providers.CartoDB.Positron,
)

In [ ]:
plot_timeseries(
    sf_types_kmeans,
    column="kmeans",
    categorical=True,
    ncols=3,
    nrows=1,
    figsize=(15, 5),
    ctxmap=ctx.providers.CartoDB.Positron,
)

In [ ]:
animate_timeseries(
    sf_types_kmeans,
    column="kmeans",
    categorical=True,
    filename="sf_kmeans.gif",
    figsize=(12, 8),
    fps=3,
    ctxmap=ctx.providers.CartoDB.Positron,
)

In [ ]:
from IPython.display import Image

In [ ]:
Image("sf_kmeans.gif")

In [ ]:
spmk = transition(sf_types_kmeans, cluster_col="kmeans")

In [ ]:
type(spmk)

In [ ]:
spmk.p

In [ ]:
spmk.LR_p_value  # strong evidence of spatial dependence in transition dynamics

In [ ]:
plot_transition_matrix(sf_types_kmeans, cluster_col="kmeans")

## Accessibility Isochrones

In [ ]:
import pandana as pdna

https://github.com/udst/pandana

In [ ]:
sfnet = pdna.Network.from_hdf5("data/41860.h5")

In [ ]:
from geosnap.analyze import isochrones_from_id, isochrones_from_gdf

In [ ]:
from geopandas.tools import geocode

In [ ]:
import geopy

geopy.geocoders.options.default_user_agent = "geosnap-demo"

In [ ]:
berkeley = geocode(
    "university of california, berkeley",
    provider="Nominatim",
    user_agent="geosnap_v2",
    timeout=None,
)

In [ ]:
berkeley.explore()

In [ ]:
isochrones_from_id?

In [ ]:
# must provide the nearest node_id to the isochrone function
sfnet.get_node_ids(berkeley.geometry.x, berkeley.geometry.y)

In [ ]:
iso_berkeley = isochrones_from_id(
    sfnet.get_node_ids(berkeley.geometry.x, berkeley.geometry.y)[0],
    sfnet,
    [500, 1000, 2000, 5000],
)

In [ ]:
iso_berkeley.explore()

In [ ]:
isochrones?

In [ ]:
schools = datasets.nces(dataset="schools")

In [ ]:
schools.plot()

In [ ]:
sf_schools = schools[schools.intersects(census.to_crs(schools.crs).unary_union)]

In [ ]:
sf_schools.plot()

In [ ]:
iso_2km = isochrones_from_gdf(sf_schools.head(20), network=sfnet, threshold=2000)

In [ ]:
m = sf_schools.head(20).explore()
iso_2km.explore(m=m)